# Installazione Spark e altro

Per iniziare ho installato nell'ambiente Spark, Parselmouth e ho montato il drive su cui si trovano i file audio mp3 e il documento di testo con le trascrizioni.

In [1]:
#Installazione spark

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
!tar xf spark-3.5.5-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"
import findspark
findspark.init()
import pyspark
print(pyspark.version)
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc=spark.sparkContext

--2025-03-06 14:25:12--  https://downloads.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400724056 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.5-bin-hadoop3.tgz’

spark-3.5.5-bin-had 100%[===================>] 382.16M  12.8MB/s    in 30s     

2025-03-06 14:25:43 (12.6 MB/s) - ‘spark-3.5.5-bin-hadoop3.tgz’ saved [400724056/400724056]

<module 'pyspark.version' from '/content/spark-3.5.5-bin-hadoop3/python/pyspark/version.py'>


In [2]:
#Installazione parselmouth

!pip install praat-parselmouth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 17.6 MB/s eta 0:00:00


In [3]:
#Montaggio drive con audio e trascrizioni

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Funzioni utili

In seguito ho creato 3 funzioni per estrarre le feature che servono per comporre i samples e una funzione che estragga le trascrizioni dal file di testo e le inserisca in un dizionario.

In [4]:
import librosa
import librosa.feature
import numpy as np
import parselmouth
from parselmouth.praat import call
from sklearn.feature_extraction.text import TfidfVectorizer

# Funzione per estrarre feature audio con librosa
def extract_audio_features(audio_path, sr=22050, n_mfcc=13):
    y, sr = librosa.load(audio_path, sr=sr)

    # Estrae MFCC
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs, axis=1)  # Media per stabilizzare i dati

    # Estrae RMSE (Energy)
    rms = librosa.feature.rms(y=y)
    rms_mean = np.mean(rms)

    return np.concatenate((mfccs_mean, [rms_mean]))

# Funzione per estrarre il pitch con Parselmouth
def extract_pitch(audio_path):
    snd = parselmouth.Sound(audio_path)
    pitch = call(snd, "To Pitch", 0.0, 75, 600)
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz")  # Media del pitch

    return np.array([mean_pitch])

# Funzione per ottenere la rappresentazione testuale con TF-IDF
def extract_text_features(text, vectorizer):
    return vectorizer.transform([text]).toarray()[0]

# Funzione per estrarre le trascrizioni dal file txt
def load_transcriptions(txt_file):
    transcriptions = {}
    with open(txt_file, "r", encoding="utf-8") as f:
        lines = f.read().strip().split("\n\n")  # Divide il file in blocchi separati da una riga vuota

    for block in lines:
        lines = block.split("\n")  # Ogni blocco contiene nome file + trascrizione
        if len(lines) >= 2:
            filename = lines[0].strip()  # Prima riga: nome del file audio
            transcript = " ".join(lines[1:]).strip()  # Tutto il resto è la trascrizione
            transcriptions[filename] = transcript  # Aggiungi al dizionario

    return transcriptions

# Creazione Samples completi (Librosa + Parselmouth + TF-IDF)

In questa fase ho creato i samples completi e li ho salvati in un file pkl sempre sul drive (il file si trova anche nella repository).

Ricordo che la cartella con tutti gli audio e le trascrizioni si chiama "audiozzi".

In [12]:
import pandas as pd

# Percorsi delle cartelle
audio_folder = "/content/drive/MyDrive/audiozzi"
transcriptions_file = "/content/drive/MyDrive/audiozzi/Trascrizioni.txt"

# Carica le trascrizioni su dizionario con la funzione creata prima
transcriptions = load_transcriptions(transcriptions_file)

# Crea il vettorizzatore TF-IDF e lo adatta su tutte
vectorizer = TfidfVectorizer()
vectorizer.fit(list(transcriptions.values()))

# Lista per contenere i dati finali
dataset = []

# Loop su tutti i file audio nella cartella
for file in os.listdir(audio_folder):
    if file.endswith(".mp3"):  # Filtra solo i file audio
        audio_path = os.path.join(audio_folder, file)

        # Determina l'etichetta (1 = Urgente, 0 = Normale) (sfrutto il fatto che ho rinominato i file)
        label = 1 if file.endswith("u.mp3") else 0

        # Recupera la trascrizione corrispondente
        transcript = transcriptions.get(file, None)
        if transcript is None:
            print(f"Nessuna trascrizione trovata per {file}, skippato.")    # Mi aiuta se trova errori nel file di testo
            continue

        # Estrae feature audio
        audio_features = extract_audio_features(audio_path)
        pitch_feature = extract_pitch(audio_path)

        # Estrae feature testuali
        text_features = extract_text_features(transcript, vectorizer)

        # Concatena tutte le feature
        sample = np.concatenate((audio_features, pitch_feature, text_features))

        # Salva i dati
        dataset.append({
            "filename": file,
            "features": sample,
            "transcript": transcript,
            "label": label
        })

# Converte in DataFrame per salvataggio
df = pd.DataFrame(dataset)
df.to_pickle("/content/drive/MyDrive/audiozzi/samplesCompleti.pkl")  # Salva in formato binario
#df.to_csv("/content/drive/MyDrive/audiozzi/prepared_dataset.csv", index=False)

print(f"{len(df)} campioni salvati.")

300 campioni salvati.


# Test dei modelli

In questa fase ho recuperato i samples dal file binario (non è necessario se li abbiamo appena estratti nello stesso notebook).

Ho diviso il dataset in training e test set (80-20).

Ho applicato 3 modelli: Random Forest, Logistic Regression e Gradient Boosting Trees

In [13]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import DenseVector
from pyspark.sql import Row

# Carica il dataset, pescando il file binario
df = pd.read_pickle("/content/drive/MyDrive/audiozzi/samplesCompleti.pkl")

# Converte in un DataFrame Spark con DenseVector
spark_df = spark.createDataFrame([
    Row(filename=row["filename"],
        features=DenseVector(row["features"]),  # Converti in DenseVector
        label=int(row["label"]))  # Controllo label int
    for _, row in df.iterrows()
])

In [14]:
#Divido training e test set in proporzione 80-20

train_df, test_df = spark_df.randomSplit([0.8, 0.2], seed=42)

In [15]:
#Primo modello: Random Forest

from pyspark.ml.classification import RandomForestClassifier

# Definisce il modello di Random Forest
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=50)

# Addestra il modello sui dati di training
model = rf.fit(train_df)

# Genera le previsioni sul dataset di test
predictions = model.transform(test_df)

#predictions.select("features", "label", "prediction").show(10)

# Definisce l'evaluator per l'accuratezza
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# Calcola l'accuratezza
accuracy = evaluator.evaluate(predictions)
print(f"Accuratezza del modello Random Forest: {accuracy * 100:.2f}%")

Accuratezza del modello Random Forest: 98.08%


In [16]:
#Secondo modello: Logistic regression

from pyspark.ml.classification import LogisticRegression

# Definisce il modello di regressione logistica
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Addestra il modello
lr_model = lr.fit(train_df)

# Fa le previsioni
lr_predictions = lr_model.transform(test_df)

# Valutazione
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_accuracy = evaluator.evaluate(lr_predictions)
print(f"Accuratezza Logistic Regression: {lr_accuracy * 100:.2f}%")

Accuratezza Logistic Regression: 100.00%


In [17]:
#terzo modello: gradient boosting trees

from pyspark.ml.classification import GBTClassifier

# Definisc il modello Gradient Boosted Trees
gbt = GBTClassifier(featuresCol="features", labelCol="label", maxIter=50)

# Addestra il modello
gbt_model = gbt.fit(train_df)

# Fa le previsioni
gbt_predictions = gbt_model.transform(test_df)

# Valutazione
gbt_accuracy = evaluator.evaluate(gbt_predictions)
print(f"Accuratezza Gradient-Boosted Trees: {gbt_accuracy * 100:.2f}%")

Accuratezza Gradient-Boosted Trees: 96.15%
